In [14]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from skimage.exposure import exposure
from skimage.feature import hog

import numpy as np
import matplotlib.pyplot as plt


# Downloading Data

In [20]:
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.Grayscale(num_output_channels=1)
])

# Loading Datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# Creating DataLoaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False, num_workers=2)
print('DataLoaders Are Ready.')

Files already downloaded and verified
Files already downloaded and verified
DataLoaders Are Ready.


# HOG Feature Extraction

In [22]:
# HOG Parameters
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
block_norm = 'L2-Hys'


class HOGFeatureExtractor(nn.Module):
    def __init__(self):
        super(HOGFeatureExtractor).__init__()
        
    def forward(self, x):
        hog_features_list = []        
        # Iterating over data in each batch and extract hog features 
        for image in x:
            # Convert Image to numpy for hog
            np_image = image.numpy().squeeze()
            hog_features = hog(
                np_image,
                orientations=orientations,
                pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block,
                block_norm=block_norm
            )
            hog_features_list.append(hog_features)
        return torch.tensor(hog_features_list, dtype=torch.float32)


In [ ]:
class Net(nn.Module):
    